In [1]:
from config import Config
from data_module import DataModule
from model_module import ModelModule
from trainer_module import TrainerModule
from upload_results import save_readme
from huggingface_hub import login, create_repo

c:\Users\aswat\Desktop\sep_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("🔧 Loading configuration...")
cfg = Config()

🔧 Loading configuration...


In [3]:
print("📂 Preparing data...")
data_module = DataModule(cfg)
train_ds, eval_ds, test_ds, data_collator = data_module.prepare()
print(f"✅ Train size: {len(train_ds)}, Eval size: {len(eval_ds)}, Test size: {len(test_ds)}")

📂 Preparing data...


2025-09-09 22:05:09,852 - data_module - INFO - 📥 Loading dataset: sreejith8100/Sp1786_multiclass-sentiment-analysis-dataset ...
c:\Users\aswat\Desktop\sep_project\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aswat\.cache\huggingface\hub\datasets--sreejith8100--Sp1786_multiclass-sentiment-analysis-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/e

✅ Train size: 31232, Eval size: 5205, Test size: 5205


In [4]:
print("🤖 Loading model...")
model_module = ModelModule(cfg)
model = model_module.load_model()
print(f"✅ Model loaded on device: {model.device}")

2025-09-09 22:05:29,212 - model_module - INFO - Loading model: distilbert-base-uncased


🤖 Loading model...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded on device: cuda:0


In [5]:
print("🎯 Setting up Trainer...")
trainer_module = TrainerModule(model, data_module.tokenizer, cfg)
trainer = trainer_module.get_trainer(train_ds, eval_ds, data_collator)

🎯 Setting up Trainer...


c:\Users\aswat\Desktop\sep_project\core\trainer_module.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(


In [6]:
print("🚀 Starting training...")
trainer.train()
print("✅ Training finished")

🚀 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.581800,0.580448,0.759654,0.760114
2,0.487300,0.579571,0.762536,0.763410
3,0.385400,0.633098,0.760999,0.761890


✅ Training finished


In [ ]:
print("📊 Evaluating model on validation set...")
metrics = trainer.evaluate()
print(f"✅ Evaluation results: {metrics}")

In [ ]:
print("🧪 Running inference on test set...")
test_metrics = trainer.evaluate(eval_dataset=test_ds)
print(f"✅ Test results: {test_metrics}")
print("🏁 Training pipeline complete.")

# 🔥 Save README with results
save_readme(cfg, metrics, test_metrics)

In [ ]:
if cfg.push_to_hub:
      print("🔑 Logging into Hugging Face Hub...")
      login(token=cfg.hub_token)
      print("⬆️ Preparing Hugging Face Hub repo...")
      create_repo(cfg.repo_id, exist_ok=True)  # auto-create if missing
      print("⬆️ Pushing model to Hugging Face Hub...")
      trainer.push_to_hub(cfg.repo_id)
      print("✅ Model pushed to hub")